# ANFIS ANALYSIS AND COMPARISON WITH ANN

I develop a Multiclass ANFIS and compare it with ANN to predict the presence of Heart Disease in a patient.

Predicted values ranges from 0 to 4. 0 being no presence of Heart Disease and 1,2,3,4 are the stages of Heart Disease, as seen in the analysis below.

`Data source` : https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/

`ANFIS implementation code`: https://github.com/gregorLen/AnfisTensorflow2.0

In [ ]:
from sklearn.model_selection import train_test_split 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#scaler
from sklearn.preprocessing import MinMaxScaler
from Models import myanfis
import sys
sys.maxsize
# KFold 
from sklearn.model_selection import KFold
import tensorflow as tf

In [ ]:
# import data
data = pd.read_csv('heart_data.csv')

# read data
data.head()

In [ ]:
# replacing ? with NaN
for i in data.columns:
    data[i]=data[i].replace('?', np.nan)

In [ ]:
pd.value_counts(data['region'])

In [ ]:
# all variables are numeric
data = data.loc[:, data.columns != 'region'].apply(pd.to_numeric)

In [ ]:
data.describe().transpose()

In [ ]:
pd.value_counts(data['heart_disease']).plot(kind= 'bar').set_title('Target variable Class Distribution')

In [ ]:
data.info()

In [ ]:
# checking missing observations
import missingno

missingno.bar(data, color= 'green')

## cleaning the data

- we cannot drop all missing observations.
- we thus fill in with the mean of each column, based on age group

In [ ]:
#creating age group variable
# Reference: LHENNY, 2021
# https://www.kaggle.com/code/lhenny/simple-data-cleansing

#define
def age_group (age):
    if age <= 35:
        return 'youth'
    if 36 <= age <= 45:
        return 'adult'
    if 46 <= age <= 60:
        return 'senior'
    else:
        return 'elderly'

In [ ]:
#creating variable 'age group'
#apply conversion
data['age_group'] = data['age'].apply(age_group)

#order
age_group_classes = ['youth', 'adult', 'senior', 'elderly']

#convert to factor variable
from pandas.api.types import CategoricalDtype

age_group_convert = CategoricalDtype(categories=age_group_classes, ordered = True)
data['age_group'] = data['age_group'].astype(age_group_convert)

In [ ]:
#filling missing observations for trestbps in each age group
#we use mean for each age group to fill missing trestbps for each group

#mean for each age group
trestbps_mean =  data.groupby('age_group')['trestbps'].mean()
#round to 1 d.p
trestbps_mean = trestbps_mean.apply(lambda x: round(x,1))
#fill trestbps for youth group
data.loc[(data['age_group'] == 'youth'), 'trestbps'] = data.loc[(data['age_group']=='youth'),'trestbps'].fillna(trestbps_mean.loc['youth'])
#fill trestbps for adult group
data.loc[(data['age_group'] == 'adult'), 'trestbps'] = data.loc[(data['age_group']=='adult'),'trestbps'].fillna(trestbps_mean.loc['adult'])
#fill trestbps for senior group
data.loc[(data['age_group'] == 'senior'), 'trestbps'] = data.loc[(data['age_group']=='senior'),'trestbps'].fillna(trestbps_mean.loc['senior'])
#fill trestbps for elderly group
data.loc[(data['age_group'] == 'elderly'), 'trestbps'] = data.loc[(data['age_group']=='elderly'),'trestbps'].fillna(trestbps_mean.loc['elderly'])


#filling missing observations for chol in each age group
#we use mean for each age group to fill missing chol for each group

#mean for each age group
chol_mean =  data.groupby('age_group')['chol'].mean()
#round to 1 d.p
chol_mean = chol_mean.apply(lambda x: round(x,1))
#fill chol for youth group
data.loc[(data['age_group'] == 'youth'), 'chol'] = data.loc[(data['age_group']=='youth'),'chol'].fillna(chol_mean.loc['youth'])
#fill chol for adult group
data.loc[(data['age_group'] == 'adult'), 'chol'] = data.loc[(data['age_group']=='adult'),'chol'].fillna(chol_mean.loc['adult'])
#fill chol for senior group
data.loc[(data['age_group'] == 'senior'), 'chol'] = data.loc[(data['age_group']=='senior'),'chol'].fillna(chol_mean.loc['senior'])
#fill chol for elderly group
data.loc[(data['age_group'] == 'elderly'), 'chol'] = data.loc[(data['age_group']=='elderly'),'chol'].fillna(chol_mean.loc['elderly'])


#filling missing observations for fbs in each age group
#we use mean for each age group to fill missing fbs for each group

#mean for each age group
fbs_mean =  data.groupby('age_group')['fbs'].mean()
#round to 1 d.p
fbs_mean = fbs_mean.apply(lambda x: round(x,1))
#fill fbs for youth group
data.loc[(data['age_group'] == 'youth'), 'fbs'] = data.loc[(data['age_group']=='youth'),'fbs'].fillna(fbs_mean.loc['youth'])
#fill fbs for adult group
data.loc[(data['age_group'] == 'adult'), 'fbs'] = data.loc[(data['age_group']=='adult'),'fbs'].fillna(fbs_mean.loc['adult'])
#fill fbs for senior group
data.loc[(data['age_group'] == 'senior'), 'fbs'] = data.loc[(data['age_group']=='senior'),'fbs'].fillna(fbs_mean.loc['senior'])
#fill fbs for elderly group
data.loc[(data['age_group'] == 'elderly'), 'fbs'] = data.loc[(data['age_group']=='elderly'),'fbs'].fillna(fbs_mean.loc['elderly'])


#filling missing observations for restecg in each age group
#we use mean for each age group to fill missing restecg for each group

#mean for each age group
restecg_mean =  data.groupby('age_group')['restecg'].mean()
#round to 1 d.p
restecg_mean = restecg_mean.apply(lambda x: round(x,1))
#fill restecg for youth group
data.loc[(data['age_group'] == 'youth'), 'restecg'] = data.loc[(data['age_group']=='youth'),'restecg'].fillna(restecg_mean.loc['youth'])
#fill restecg for adult group
data.loc[(data['age_group'] == 'adult'), 'restecg'] = data.loc[(data['age_group']=='adult'),'restecg'].fillna(restecg_mean.loc['adult'])
#fill restecg for senior group
data.loc[(data['age_group'] == 'senior'), 'restecg'] = data.loc[(data['age_group']=='senior'),'restecg'].fillna(restecg_mean.loc['senior'])
#fill restecg for elderly group
data.loc[(data['age_group'] == 'elderly'), 'restecg'] = data.loc[(data['age_group']=='elderly'),'restecg'].fillna(restecg_mean.loc['elderly'])


#filling missing observations for thalach in each age group
#we use mean for each age group to fill missing thalach for each group

#mean for each age group
thalach_mean =  data.groupby('age_group')['thalach'].mean()
#round to 1 d.p
thalach_mean = thalach_mean.apply(lambda x: round(x,1))
#fill thalach for youth group
data.loc[(data['age_group'] == 'youth'), 'thalach'] = data.loc[(data['age_group']=='youth'),'thalach'].fillna(thalach_mean.loc['youth'])
#fill thalach for adult group
data.loc[(data['age_group'] == 'adult'), 'thalach'] = data.loc[(data['age_group']=='adult'),'thalach'].fillna(thalach_mean.loc['adult'])
#fill thalach for senior group
data.loc[(data['age_group'] == 'senior'), 'thalach'] = data.loc[(data['age_group']=='senior'),'thalach'].fillna(thalach_mean.loc['senior'])
#fill thalach for elderly group
data.loc[(data['age_group'] == 'elderly'), 'thalach'] = data.loc[(data['age_group']=='elderly'),'thalach'].fillna(thalach_mean.loc['elderly'])


#filling missing observations for exang in each age group
#we use mean for each age group to fill missing exang for each group

#mean for each age group
exang_mean =  data.groupby('age_group')['exang'].mean()
#round to 1 d.p
exang_mean = exang_mean.apply(lambda x: round(x,1))
#fill exang for youth group
data.loc[(data['age_group'] == 'youth'), 'exang'] = data.loc[(data['age_group']=='youth'),'exang'].fillna(exang_mean.loc['youth'])
#fill exang for adult group
data.loc[(data['age_group'] == 'adult'), 'exang'] = data.loc[(data['age_group']=='adult'),'exang'].fillna(exang_mean.loc['adult'])
#fill exang for senior group
data.loc[(data['age_group'] == 'senior'), 'exang'] = data.loc[(data['age_group']=='senior'),'exang'].fillna(exang_mean.loc['senior'])
#fill exang for elderly group
data.loc[(data['age_group'] == 'elderly'), 'exang'] = data.loc[(data['age_group']=='elderly'),'exang'].fillna(exang_mean.loc['elderly'])


#filling missing observations for oldpeak in each age group
#we use mean for each age group to fill missing oldpeak for each group

#mean for each age group
oldpeak_mean =  data.groupby('age_group')['oldpeak'].mean()
#round to 1 d.p
oldpeak_mean = oldpeak_mean.apply(lambda x: round(x,1))
#fill oldpeak for youth group
data.loc[(data['age_group'] == 'youth'), 'oldpeak'] = data.loc[(data['age_group']=='youth'),'oldpeak'].fillna(oldpeak_mean.loc['youth'])
#fill oldpeak for adult group
data.loc[(data['age_group'] == 'adult'), 'oldpeak'] = data.loc[(data['age_group']=='adult'),'oldpeak'].fillna(oldpeak_mean.loc['adult'])
#fill oldpeak for senior group
data.loc[(data['age_group'] == 'senior'), 'oldpeak'] = data.loc[(data['age_group']=='senior'),'oldpeak'].fillna(oldpeak_mean.loc['senior'])
#fill oldpeak for elderly group
data.loc[(data['age_group'] == 'elderly'), 'oldpeak'] = data.loc[(data['age_group']=='elderly'),'oldpeak'].fillna(oldpeak_mean.loc['elderly'])


#filling missing observations for slope in each age group
#we use mean for each age group to fill missing slope for each group

#mean for each age group
slope_mean =  data.groupby('age_group')['slope'].mean()
#round to 1 d.p
slope_mean = slope_mean.apply(lambda x: round(x,1))
#fill slope for youth group
data.loc[(data['age_group'] == 'youth'), 'slope'] = data.loc[(data['age_group']=='youth'),'slope'].fillna(slope_mean.loc['youth'])
#fill slope for adult group
data.loc[(data['age_group'] == 'adult'), 'slope'] = data.loc[(data['age_group']=='adult'),'slope'].fillna(slope_mean.loc['adult'])
#fill slope for senior group
data.loc[(data['age_group'] == 'senior'), 'slope'] = data.loc[(data['age_group']=='senior'),'slope'].fillna(slope_mean.loc['senior'])
#fill slope for elderly group
data.loc[(data['age_group'] == 'elderly'), 'slope'] = data.loc[(data['age_group']=='elderly'),'slope'].fillna(slope_mean.loc['elderly'])


#filling missing observations for ca in each age group
#we use mean for each age group to fill missing ca for each group

#mean for each age group
ca_mean =  data.groupby('age_group')['ca'].mean()
#round to 1 d.p
ca_mean = ca_mean.apply(lambda x: round(x,1))
#fill ca for youth group
data.loc[(data['age_group'] == 'youth'), 'ca'] = data.loc[(data['age_group']=='youth'),'ca'].fillna(ca_mean.loc['youth'])
#fill ca for adult group
data.loc[(data['age_group'] == 'adult'), 'ca'] = data.loc[(data['age_group']=='adult'),'ca'].fillna(ca_mean.loc['adult'])
#fill ca for senior group
data.loc[(data['age_group'] == 'senior'), 'ca'] = data.loc[(data['age_group']=='senior'),'ca'].fillna(ca_mean.loc['senior'])
#fill ca for elderly group
data.loc[(data['age_group'] == 'elderly'), 'ca'] = data.loc[(data['age_group']=='elderly'),'ca'].fillna(ca_mean.loc['elderly'])


#filling missing observations for thal in each age group
#we use mean for each age group to fill missing thal for each group

#mean for each age group
thal_mean =  data.groupby('age_group')['thal'].mean()
#round to 1 d.p
thal_mean = thal_mean.apply(lambda x: round(x,1))
#fill thal for youth group
data.loc[(data['age_group'] == 'youth'), 'thal'] = data.loc[(data['age_group']=='youth'),'thal'].fillna(thal_mean.loc['youth'])
#fill thal for adult group
data.loc[(data['age_group'] == 'adult'), 'thal'] = data.loc[(data['age_group']=='adult'),'thal'].fillna(thal_mean.loc['adult'])
#fill thal for senior group
data.loc[(data['age_group'] == 'senior'), 'thal'] = data.loc[(data['age_group']=='senior'),'thal'].fillna(thal_mean.loc['senior'])
#fill thal for elderly group
data.loc[(data['age_group'] == 'elderly'), 'thal'] = data.loc[(data['age_group']=='elderly'),'thal'].fillna(thal_mean.loc['elderly'])

In [ ]:
# confirming that all variables are filled, without missing observations
missingno.bar(data, color= 'green')

In [ ]:
data = data.drop('age_group', axis= 1)

In [ ]:
pd.value_counts(data['heart_disease'])

## Modelling - ANFIS

In [ ]:
'''it is good to make predictions simpler, as either with heart attack or not.
we make prediction into binary only, with the following conditions:
1 - 4
0 - has no attack when the value is 0 has attack, when the values are from 1 to

This helps also in overfitting since class 2 to 4 are minor in count'''

def binarizer(x):
    if x == 0:
        return 0
    else: return 1
    
data['target'] = data['heart_disease'].apply(binarizer)

pd.value_counts(data['target']).plot(kind= 'bar').set_title('Distribution of Classes in the Target variable');

'''
it is seen that the classes are now approximately evenly balanced, and the risk of overfitting is solved
'''

In [ ]:
data = data.drop('heart_disease', axis=1)

In [ ]:
# scaling
minmaxScaler = MinMaxScaler()

data2 = data.copy()

# inputs = [3,4]
# target = heart_disease
data2['trestbps'] = minmaxScaler.fit_transform(data2[['trestbps']])
data2['chol'] = minmaxScaler.fit_transform(data2[['chol']])

In [ ]:
'''
the ANFIS algorithm used is from Gregor Len, link: https://github.com/gregorLen/AnfisTensorflow2.0
The algorithm is based on Tensorflow 2.0., currently supporting two types of membership function:
 -gaussian (used in this project)
 -generalized bell
 
Dependencies:
 -Dependencies
 -Python 3.6-3.9
 -tensorflow 2.6.0
 -numpy
 -pandas
 -sklearn
 -matplotlib
 -seaborn
 
However, the implementation is still a work in progress since only 6 inputs are taken.

'''

In [ ]:
# training and validation sets
#training
X = data2.iloc[:-120,[3,4,5,6,8,9]]
Y = data2.iloc[:-120,-1]

In [ ]:
# model parameters
param = myanfis.fis_parameters(
        n_input = 6,                # the number of Regressors
        n_memb = 4,                 # the number of fuzzy memberships
        batch_size = 5,           
        memb_func = 'gaussian',      # 'gaussian' / 'gbellmf' / 'sigmoid'
        optimizer = 'sgd',          # sgd / adam / ...
        loss = tf.keras.losses.MeanAbsoluteError(),               # mse / mae / huber_loss / mean_absolute_percentage_error / ...
        n_epochs = 15               # 10 / 25 / 50 / 100 / ...
        )

In [ ]:
# We set KFold = 2 because this task is a supervised method
kfold = KFold(n_splits=2)
histories = []

# We got the necessary indexes for our folds with kfold.split and got them as X_train, X_test, Y_train, Y_test
# and we carried out the Fold Fold training
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    # We call ANFIS generated from Tensorflow.Keras models with the required parameters
    fis = myanfis.ANFIS(n_input = param.n_input,
                    n_memb = param.n_memb,
                    batch_size = param.batch_size,
                    memb_func = param.memb_func,
                    name = 'myanfis' # 
                    )

    # We have compiled our model with the following parameters
    fis.model.compile(optimizer=param.optimizer,
                      loss=param.loss,
                      metrics=['mae', 'mse']  # ['mae', 'mse']
                      )
    # here we start with the training of our model
    # then assigned the training results of the model to history
    history = fis.fit(X_train, Y_train,
                  epochs=param.n_epochs,
                  batch_size=param.batch_size,
                  validation_data = (X_test, Y_test),
                  )
    # Since the model will train fold fold, we kept the training results in a list called histories.
    histories.append(history)

In [ ]:
y_pred = fis.model.predict(X_test, batch_size= 5)

In [ ]:

'''
The predictions made by ANFIS are not binary (not 0 and 1, but continous), and thus,
we  have to set the prediction to binary so we can derive the evaluation metrics

'''
y_pred_toformat = pd.DataFrame(y_pred)
y_pred_toformat.insert(0, 'prediction_ID', range(1, 1 + len(y_pred_toformat)))
y_pred_toformat = y_pred_toformat.rename(columns={0: "anfis_predictions"})


# formatter
def ypred_formatter(r):
    if r <= 0:
        return 0
    else: return 1

In [ ]:
y_pred_toformat['anfis_binary_predictions'] = y_pred_toformat['anfis_predictions'].apply(ypred_formatter)

In [ ]:
anfis_y_pred = np.array(y_pred_toformat['anfis_binary_predictions'])

In [ ]:
# confusion matrix
plt.figure(figsize=(8,10))
from sklearn.metrics import confusion_matrix
import seaborn as sns
conf_mat = confusion_matrix(Y_test, anfis_y_pred)
sns.heatmap(conf_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False)

In [ ]:
# classification report
from sklearn.metrics import classification_report
print(classification_report(Y_test, anfis_y_pred))

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
print('The accuracy score of ANFIS is',accuracy_score(Y_test, anfis_y_pred)*100, '%')

In [ ]:
'''
from above, the True positive and False negative are well predicted.
the magnitude of true negative and false positives are low (67 and 37 observations)
'''

In [ ]:
fis.plotmfs()

In [ ]:
pd.DataFrame(histories[0].history).plot()

# loss = mae
# val_loss = val_mae

In [ ]:
history_df = pd.DataFrame(history.history)

plt.plot(history_df.loc[:, ['loss']], "#6daa9f", label='Training loss')
plt.plot(history_df.loc[:, ['val_loss']],"#774571", label='Validation loss')
plt.title('ANFIS Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(loc="best")

plt.show()

## Modelling - ANN (For comparison with ANFIS)

In [ ]:
from keras.layers import Dense, BatchNormalization, Dropout
from keras.models import Sequential
from keras import callbacks
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.131, random_state=2023)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Dropout
model = tf.keras.Sequential([
    layers.Dense(20, activation='relu', name='layer1'),
    Dropout(0.2),
    layers.Dense(25, activation='relu', name='layer2'),
    Dropout(0.5),
    layers.Dense(10, activation='relu', name='layer3'),
    layers.Dense(2, activation='sigmoid', name='f-layer'),
])

In [ ]:
from tensorflow import keras 
model.compile(
      loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
      optimizer = keras.optimizers.Adam(lr = 0.001),
      metrics = ['mae', 'mse']
)

In [ ]:
model.fit(x_train, y_train, batch_size = 5, epochs = 15, verbose=2)

In [ ]:
ann_y_pred = model.predict(X_test, batch_size=5)

'''
the y predictions account for classes 0 and 1 seperately, and thus, if the values for 0 are greater than those of 1, then the class prediction is 0, otherwise, class 1.
'''
ann_y_pred_df = pd.DataFrame(ann_y_pred)

In [ ]:
def class_determiner (t):
    if t < 0.5:
        return 1
    else: return 0
    
# apply
ann_y_pred_df['ann_binary_predictions'] = ann_y_pred_df[0].apply(class_determiner)

In [ ]:
ann_y_pred_df

In [ ]:
ann_y_pred = np.array(ann_y_pred_df['ann_binary_predictions'])

In [ ]:
# confusion matrix
plt.figure(figsize=(8,10))
from sklearn.metrics import confusion_matrix
import seaborn as sns
conf_mat = confusion_matrix(Y_test, ann_y_pred)
sns.heatmap(conf_mat, square=True, annot=True, cmap='Blues', fmt='d', cbar=False)

In [ ]:
# classification report
print(classification_report(Y_test, ann_y_pred))

In [ ]:
# accuracy
from sklearn.metrics import accuracy_score
print('The accuracy score of ANFIS is',accuracy_score(Y_test, ann_y_pred)*100, '%')

In [ ]:
'''
surprisingly, ANN has performed better than the ANFIS classifier
Other (and most) studies have also found out ANN to be more accurate than ANFIS.
This is beacuse the ANFIS model is seen to overfit the data even when tuned.
The ANN is a better classifier.
'''

In [ ]:
pd.DataFrame(histories[0].history).plot().set_title("ANN Loss, MAE, MSE, Valuation Loss")

# loss = mae
# val_loss = val_mae

## END